<a href="https://colab.research.google.com/github/leehon94/Automated-Product-Identification_PJS/blob/master/Keras%20Modelle/Xception5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
from google.colab import drive 
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras import optimizers
from keras import layers
from keras import models
from keras.preprocessing import image
from keras import backend as K

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


base_dir = r'/content/gdrive/My Drive/MVP Datensatz/'
train_dir = r'/content/gdrive/My Drive/MVP Datensatz/Trainingsdatensatz5'


In [0]:


#Beispiel für mögliche Data Augmentations


In [0]:


from keras.applications import Xception
base_model = Xception(weights='imagenet',
  include_top=False,
  pooling='avg',
  input_shape=(299,299,3)) # input_shape = 299x299x3 by default

#datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
#batch_size = 20

train_datagen = ImageDataGenerator(featurewise_center=False,
                          samplewise_center=False,
                          featurewise_std_normalization=False,
                          samplewise_std_normalization=False,
                          zca_whitening=False,
                          zca_epsilon=1e-06,
                          rotation_range=3,
                          width_shift_range=[-1, 0, +1],
                          height_shift_range=[-1, 0, +1],
                          brightness_range=None, 
                          shear_range=0.1, 
                          zoom_range=0.2,
                          channel_shift_range=0.0, 
                          fill_mode='nearest', 
                          cval=0.0,
                          horizontal_flip=False, 
                          vertical_flip=False, 
                          rescale=1./255, 
                          preprocessing_function=None, 
                          data_format='channels_last', 
                          validation_split=0.2, 
                          interpolation_order=1, 
                          dtype='float32')

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(299,299),
    batch_size=10,
    class_mode='categorical',
    subset='training')

valid_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(299,299),
    batch_size=10,
    class_mode='categorical',
    subset='validation')


                      

Found 80 images belonging to 5 classes.
Found 20 images belonging to 5 classes.


In [0]:
base_model.summary()

In [0]:
batch_size = 10
def extract_features(directory, sample_count, gen):
  features = np.zeros(shape=(sample_count, 2048)) 
  labels = np.zeros(shape=(sample_count, 5))
  generator = gen
  i=0
  for inputs_batch, labels_batch in generator:
    features_batch = base_model.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
      break
  return features, labels
train_features, train_labels = extract_features(train_dir, 80, train_generator)
valid_features, valid_labels = extract_features(train_dir, 20, valid_generator)

In [0]:
model = models.Sequential()
model.add(base_model)
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))

In [0]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False


# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(train_generator,
  epochs=50,
  validation_data = valid_generator)

Epoch 1/50
8/8 [==============================] - 4s 542ms/step - loss: 1.5318 - acc: 0.3500 - val_loss: 1.1610 - val_acc: 0.4000
Epoch 2/50
8/8 [==============================] - 2s 221ms/step - loss: 1.2724 - acc: 0.5500 - val_loss: 1.1572 - val_acc: 0.7500
Epoch 3/50
8/8 [==============================] - 2s 309ms/step - loss: 1.0781 - acc: 0.6625 - val_loss: 0.9710 - val_acc: 0.7000
Epoch 4/50
8/8 [==============================] - 2s 310ms/step - loss: 1.0081 - acc: 0.7375 - val_loss: 0.8677 - val_acc: 0.7500
Epoch 5/50
8/8 [==============================] - 2s 311ms/step - loss: 0.8605 - acc: 0.7500 - val_loss: 1.0930 - val_acc: 0.8000
Epoch 6/50
8/8 [==============================] - 3s 314ms/step - loss: 0.8727 - acc: 0.7125 - val_loss: 0.7717 - val_acc: 0.9000
Epoch 7/50
8/8 [==============================] - 3s 313ms/step - loss: 0.6786 - acc: 0.8375 - val_loss: 0.7646 - val_acc: 0.7500
Epoch 8/50
8/8 [==============================] - 3s 313ms/step - loss: 0.7602 - acc: 0.68

In [0]:
folder_path = base_dir + 'Testbilder'
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(299, 299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)
print(classes)

NameError: ignored

In [0]:
testbild = base_dir + 'Testbilder/P1040695.JPG'
testbild = image.load_img(testbild, target_size=(299,299))
testbild = image.img_to_array(testbild)
testbild = np.expand_dims(testbild, axis=0)
predictions = model.predict(testbild)



In [0]:
train_generator.classes



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [62]:
# image folder
folder_path = base_dir + 'Testbilder/'
# path to model
model_path = '/path/to/saved/model.h5'
# dimensions of images
img_width, img_height = 299, 299


# load all images into a list
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=20)
print(classes)

[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [74]:
predict = model.predict_proba(testbild)
np.argmax(predict)


TypeError: ignored

In [0]:
model.save("/content/gdrive/My Drive/Keras Models/Xception_5-2.h5")

